<a href="https://colab.research.google.com/github/ishandahal/stats453-deep_learning_torch/blob/main/Rnn/simple_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using simple RNN for sentiment analysis

### Imports

In [1]:
import torch
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
import time 
import random

torch.backends.cudnn.deterministic = True

### General Settings 

In [2]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 1e-4
BATCH_SIZE = 128
NUM_EPOCHS = 15
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 128
HIDDEN_DIM = 256
OUTPUT_DIM = 1

## Dataset
IMDB movie review dataset

In [4]:
TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state=random.seed(RANDOM_SEED),
                                          split_ratio=0.8)
print(f"Num Train: {len(train_data)}")
print(f"Num Valid: {len(valid_data)}")
print(f"Num Test: {len(test_data)}")

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 21.7MB/s]


Num Train: 20000
Num Valid: 5000
Num Test: 25000


## Build the vocabulary based on the top "VOCABULARY_SIZE" words:

In [5]:
TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f"Vocabulary size: {len(TEXT.vocab)}")
print(f"Number of classes: {len(LABEL.vocab)}")

Vocabulary size: 20002
Number of classes: 2


In [6]:
train_loader, valid_loader, test_loader = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=DEVICE)

In [7]:
print('Train')
for batch in train_loader:
    print(f"Text matrix size: {batch.text.size()}")
    print(f"Target vector size: {batch.label.size()}")
    break

print('\nValid')
for batch in valid_loader:
    print(f"Text matrix size: {batch.text.size()}")
    print(f"Target vector size: {batch.label.size()}")
    break

print("\nTest")
for batch in test_loader:
    print(f"Text matrix size: {batch.text.size()}")
    print(f"Target vector size: {batch.label.size()}")
    break

Train
Text matrix size: torch.Size([888, 128])
Target vector size: torch.Size([128])

Valid
Text matrix size: torch.Size([62, 128])
Target vector size: torch.Size([128])

Test
Text matrix size: torch.Size([42, 128])
Target vector size: torch.Size([128])


## Model

In [9]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super(RNN, self).__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        #[sentence len, batch size] => [sentence len, batch size, embedding size]
        embedded = self.embedding(text)

        #[sentence len, batch size, embedding size] => 
        # output: [sentence len, batch size, hidden size]
        # hidden: [1, batch size, hidden size]
        output, hidden = self.rnn(embedded)

        return self.fc(hidden.squeeze(0)).view(-1)

In [10]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 64
HIDDEN_DIM = 128
OUTPUT_DIM = 1

torch.manual_seed(RANDOM_SEED)
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

## Training

In [11]:
def compute_binary_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_loader):
            logits = model(batch_data.text)
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            num_examples += batch_data.label.size(0)
            correct_pred += (predicted_labels == batch_data.label.long()).sum()
        return correct_pred.float() / num_examples * 100

In [13]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):

        # Forward and Back Prop
        logits = model(batch_data.text)
        cost = F.binary_cross_entropy_with_logits(logits, batch_data.label)
        optimizer.zero_grad()

        cost.backward()

        ## update model parameters
        optimizer.step()

        ## logging 
        if not batch_idx % 50:
            print(f"Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | "
                  f"Batch: {batch_idx:03d}/{len(train_loader):03d} | "
                  f"Cost: {cost:.4f}")
    
    with torch.set_grad_enabled(False):
        print(f"training accuracy: "
              f"{compute_binary_accuracy(model, train_loader, DEVICE):.2f}%"
              f"\nvalid accuracy: "
              f"{compute_binary_accuracy(model, valid_loader, DEVICE):.2f}%")
    
    print(f"Time elapsed: {(time.time() - start_time) / 60:.2f} min")

print(f"Total time elapsed: {(time.time() - start_time) / 60:.2f} min")
print(f"Test accuracy: {compute_binary_accuracy(model, test_loader, DEVICE):.2f}%")

Epoch: 001/015 | Batch: 000/157 | Cost: 0.7088
Epoch: 001/015 | Batch: 050/157 | Cost: 0.6932
Epoch: 001/015 | Batch: 100/157 | Cost: 0.6875
Epoch: 001/015 | Batch: 150/157 | Cost: 0.6956
training accuracy: 49.97%
valid accuracy: 50.28%
Time elapsed: 0.29 min
Epoch: 002/015 | Batch: 000/157 | Cost: 0.6895
Epoch: 002/015 | Batch: 050/157 | Cost: 0.6959
Epoch: 002/015 | Batch: 100/157 | Cost: 0.6931
Epoch: 002/015 | Batch: 150/157 | Cost: 0.6944
training accuracy: 50.05%
valid accuracy: 50.12%
Time elapsed: 0.57 min
Epoch: 003/015 | Batch: 000/157 | Cost: 0.6951
Epoch: 003/015 | Batch: 050/157 | Cost: 0.6937
Epoch: 003/015 | Batch: 100/157 | Cost: 0.7039
Epoch: 003/015 | Batch: 150/157 | Cost: 0.6932
training accuracy: 49.98%
valid accuracy: 50.80%
Time elapsed: 0.85 min
Epoch: 004/015 | Batch: 000/157 | Cost: 0.6939
Epoch: 004/015 | Batch: 050/157 | Cost: 0.6937
Epoch: 004/015 | Batch: 100/157 | Cost: 0.6907
Epoch: 004/015 | Batch: 150/157 | Cost: 0.6956
training accuracy: 50.08%
valid 

In [19]:
import spacy 

nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [20]:
print("Probability positive:")
predict_sentiment(model, "I really love this movie. This is so great!")

Probability positive:


0.5515838265419006